In [2]:
#Follow along exercises with http://neuralnetworksanddeeplearning.com/chap1.html
print("yo")

yo


Does this render in github?


Notes:
Perceptrons sufficient for universal computation because the can implement NAND. Kinda cool way to introduce them

$$w \cdot x = \sum w_j x_j$$

Exercise: NAND gate for addition
TODO: hack latex drawing here
x1 x2  a == NAND(x1,x2) b == NAND(x1,a) c== NAND(x2,a) low_bit == NAND(b,c) carry_bit== NAND(a,a)  result   base10
0  0        1            1                  1               0                   0                   00          0
0  1        1            1                  0               1                   0                   01          1
1  1        0            1                  1               0                   1                   10          2

Since Perceptrons can implment NAND, the can do this too


Exercise: Sigmoid neurons simulating perceptrons, part I 
Suppose we take all the weights and biases in a network of perceptrons, and multiply them by a positive constant, c>0. Show that the behaviour of the network doesn't change.

The activation are invariant under constant rescaling since you can write
$$w \cdot x > b $$

and $$w \cdot x >b \implies K(w \cdot x) > Kb$$

Exercise: Sigmoid neurons simulating perceptrons, part II 
Suppose we have the same setup as the last problem - a network of perceptrons. Suppose also that the overall input to the network of perceptrons has been chosen. We won't need the actual input value, we just need the input to have been fixed. Suppose the weights and biases are such that w⋅x+b≠0 for the input x to any particular perceptron in the network. Now replace all the perceptrons in the network by sigmoid neurons, and multiply the weights and biases by a positive constant c>0. Show that in the limit as c→∞ the behaviour of this network of sigmoid neurons is exactly the same as the network of perceptrons. How can this fail when w⋅x+b=0 for one of the perceptrons?

Well, sigmoid neurons is given by
$$\sigma(z) = \frac{1}{1+\exp^{-z}} $$
$$ \lim_{z->\infty} \sigma \rightarrow 0$$
$$ \lim_{z->-\infty} \sigma \rightarrow 1$$
and if $z = c w\cdot x$ then the sigmoid limit become undefined as $|c| \rightarrow \infty$ and $w \cdot x \rightarrow 0$


Exercis: There is a way of determining the bitwise representation of a digit by adding an extra layer to the three-layer network above. The extra layer converts the output from the previous layer into a binary representation, as illustrated in the figure below. Find a set of weights and biases for the new output layer. Assume that the first 3 layers of neurons are such that the correct output in the third layer (i.e., the old output layer) has activation at least 0.99, and incorrect outputs have activation less than 0.01.

In summary the 4 outputs represent the 4 bits that span the space of 0-15 decimal integers. You need the 

first bit to be active if the output is odd and inactive if even. 
second bit: if greater than 1: active if odd, inactive if even
third: if greater than 3: ditto
forth: if greater than 7 ditto

this means that the first output only weights the first two neurons of the previous layer. second output weight the 2nd two. 3rd weights 4 inputs 4-7 (indexing starting at zero). and so on....

In [3]:
import numpy as np

class Network(object):

    def __init__(self, layer_dims):
        self.num_layers = len(layer_dims)
        self.layer_dims = layer_dims
        # start at 1 b/c layer_dims include the input layer at index 0
        #biases are vectors,  i.e. matrices with of n x 1, with n == number of neurons in layer
        self.biases = [np.random.randn(y,1) for y in sizes[1:]] #randn is gaussian mean 0 var 1
        # weight are matrices of m x n connecting layers n and m where n<m, m=n+1: 
        # w^nm_jk => kth neuron in nth layer to jth in m  
        self.weights = [np.random.randn(y,x) for x, y in zip(sizes[:-1],sizes[1:])]

    def feedfoward(self,a):
        '''Return output of network given a
        '''

        for b, w in zip(self.biases,self.weights):
            activaitons = sigmoid(np.dot(w,a)+b)

        return activaitons

    def sgd(self, training_data, epochs, mini_batch_size, learning_rate, test_data=None):
        '''Train network with stochastic gradient descent.

        '''

        if test_data:
            n_test = len(test_data)

        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0,n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update(mini_batch, learning_rate)
            if test_data:
                print("epoch {}: {}/{}".format(j, self.evaluate(test_data,n_test)))
            else:
                print("epoch {} complete".format(j))


    def update(self, mini_batch, learning_rate):
        del_b = [np.zeros(b.shape) for b in self.biases]
        del_w =[np.zeros(w.shape) for w in self.weights]

        for x, y in mini_batch:
            delta_del_b, delta_del_w = self.backprop(x,y)
            del_b = [d_b + delta_d_b in zip(del_b, delta_del_b)]
            del_w = [d_w + delta_d_w in zip(del_w, delta_del_w)]
        self.weights = [w -learning_rate/len(mini_batch)*d_w for w,d_w in zip(self.weights, del_w)]
        self.biases = [b -learning_rate/len(mini_batch)*d_b for b,d_b in zip(self.biases, del_b)]
        


def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

In [ ]:
The matrix ordering of the weights above is odd at first glance:
$$w^{nm}_{jk} | m=n+1 \& j \in {dim(m)} k \in {dim{n}} $$
Why this convention?

The activations are easy to write:
$$a^{'} = \sigma(w \cdot a + b)$$

Let's see this for myself
$$ w \cdot a = \sum_{j} w_{ij} a_j $$
$$ \sigma(w \cdot a + b) = \sigma(\sum_{j} w_{ij} a_j + b_i ) $$ 

now, if i want to be able to compute the upstream activation, the index ${i}$ needs to be from the layer upstream of the ${j}$ index. With this convention, i can compute the activations of the layer by applying the sigmoid function element wise:


$$a^{m}_i = \sigma(w^n_{ij}*a^n_j + b^{m}_i}$$


Exercise: An extreme version of gradient descent is to use a mini-batch size of just 1. That is, given a training input, x, we update our weights and biases according to the rules wk→w′k=wk−η∂Cx/∂wk and bl→b′l=bl−η∂Cx/∂bl. Then we choose another training input, and update the weights and biases again. And so on, repeatedly. This procedure is known as online, on-line, or incremental learning. In online learning, a neural network learns from just one training input at a time (just as human beings do). Name one advantage and one disadvantage of online learning, compared to stochastic gradient descent with a mini-batch size of, say, 20. 

Batch size of one is easy and cheap to compute, but a single example may have significant impact on the convergence. Naively, it also seems like the ordering of the samples is important in that if you sample a couple of 'weird' events first then it may take while to learn that those events are on the tail of some distribution. Not sure how to quantify this idea...


In [7]:
# sigmoid is automatically element-wise/vectorized by use of np in the defn
sigmoid(np.array([0,1,2,3]))

array([0.5       , 0.73105858, 0.88079708, 0.95257413])